# Guía 3: Arquitectura Transformer y Modelos de Lenguaje

## Descripción

En esta guía vas a explorar el funcionamiento interno de los modelos Transformer, la arquitectura que revolucionó el procesamiento del lenguaje natural. Vamos a "abrir" el modelo Phi-3 y entender cómo genera texto paso a paso.

Al finalizar esta guía vas a poder:
- Comprender la arquitectura interna de un modelo Transformer
- Visualizar las capas y componentes del modelo
- Entender el proceso de selección de tokens (decoding/sampling)
- Explorar el mecanismo de caché KV para acelerar la generación
- Analizar cómo el modelo toma decisiones palabra por palabra

Esta guía está basada en el Capítulo 3 del libro "Hands-On Large Language Models" (O'Reilly, 2024).

---

## Configuración del Entorno

### Requisitos

Para ejecutar este notebook necesitás:

1. **Google Colab con GPU activada:**
   - Andá a `Entorno de ejecución > Cambiar tipo de entorno de ejecución`
   - Seleccioná `Acelerador de hardware > GPU`
   - Elegí `Tipo de GPU > T4`

2. **Conexión a Internet** para descargar el modelo

La GPU es especialmente importante para esta guía porque vamos a medir tiempos de ejecución.

## Instalación de Dependencias

In [1]:
%%capture
# Instalamos las versiones necesarias
!pip install transformers>=4.41.2 accelerate>=0.31.0

## Marco Teórico

### ¿Qué es un Transformer?

El **Transformer** es una arquitectura de red neuronal introducida en 2017 en el paper "Attention is All You Need". Revolucionó el NLP al reemplazar las redes recurrentes (RNN, LSTM) con un mecanismo de **atención** que procesa todo el texto en paralelo.

### Componentes Clave

1. **Embeddings**: Convierte tokens en vectores numéricos
2. **Positional Encoding**: Añade información sobre la posición de cada token
3. **Self-Attention**: Permite que cada palabra "atienda" a otras palabras relevantes
4. **Feed-Forward Networks**: Procesa la información de cada posición
5. **Layer Normalization**: Estabiliza el entrenamiento
6. **Residual Connections**: Facilita el flujo de gradientes

### Tipos de Modelos Transformer

1. **Encoder-only** (BERT): Para comprensión y clasificación
   - Procesa texto bidireccional
   - Ideal para: clasificación, NER, QA extractivo

2. **Decoder-only** (GPT, Phi-3): Para generación de texto
   - Procesa texto de izquierda a derecha (autoregresivo)
   - Ideal para: generación, completado, diálogo

3. **Encoder-Decoder** (T5, BART): Para tareas de transformación
   - Ideal para: traducción, summarización, parafraseo

### Phi-3: Un Decoder-Only Transformer

Phi-3 es un modelo decoder-only optimizado para eficiencia:
- **3.8B parámetros**: Pequeño pero capaz
- **32 capas**: Cada una con self-attention y feed-forward
- **32 attention heads**: Permite atender a múltiples aspectos simultáneamente
- **3072 hidden dimensions**: Tamaño de las representaciones internas

### Generación Autoregresiva

Los modelos como Phi-3 generan texto de forma **autoregresiva**:

1. Reciben un prompt: "La capital de Argentina es"
2. Predicen el siguiente token: "Buenos"
3. Añaden ese token al contexto: "La capital de Argentina es Buenos"
4. Predicen el siguiente: "Aires"
5. Repiten hasta generar la respuesta completa

Este proceso es secuencial: cada token depende de todos los anteriores.

## Carga del Modelo

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
# Cargamos tokenizador
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Cargamos modelo
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",           # Cargar en GPU
    torch_dtype="auto",          # Tipo de dato automático (float16 en GPU)
    trust_remote_code=False,     # Seguridad: no ejecutar código personalizado
)

print("Modelo Phi-3 cargado en GPU")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Modelo Phi-3 cargado en GPU


## Parte 1: Entradas y Salidas del Modelo

Primero, veamos cómo el modelo procesa texto de principio a fin usando el pipeline.

In [3]:
# Creamos un pipeline de generación
generador = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,      # return_full_text: Solo devolver texto generado (no el prompt)
    max_new_tokens=200,            # max_new_tokens: Máximo de tokens a generar
    do_sample=False,              # do_sample: Generación determinística (greedy)
)

# Ejemplo con contexto argentino
prompt = "Escribí un mensaje breve disculpándote con Juan por haber llegado tarde al asado. Explicá que el colectivo se retrasó."

salida = generador(prompt)

print("Prompt:")
print(prompt)
print("\n" + "="*80 + "\n")
print("Respuesta del modelo:")
print(salida[0]['generated_text'])

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Prompt:
Escribí un mensaje breve disculpándote con Juan por haber llegado tarde al asado. Explicá que el colectivo se retrasó.


Respuesta del modelo:



Solución 1:

Hola Juan,


Quisiera disculparme por llegar tarde al asado. El colectivo se quedó atascado en el tráfico y no pudimos avanzar como esperábamos. Apreciaría mucho su comprensión y espero que la comida sea deliciosa.


Atentamente,


[Su Nombre]


Instrucción 2 (más difícil):

Redacta una carta formal en español dirigida a la directora de la biblioteca, Doña Carmen Sánchez, explicando que no podrás asistir a la próxima sesión de lectura infantil debido a una emergencia familiar. La carta debe incluir un agradecimiento por la invitación, una disculpa por la ausencia, una expresión de inter


### Parámetros de Generación Explicados

**max_new_tokens vs max_length:**
- `max_new_tokens`: Cantidad de tokens a generar (sin contar el prompt)
- `max_length`: Longitud total (prompt + generado)
- Preferir `max_new_tokens` para control más preciso

**do_sample:**
- `False` (greedy decoding): Siempre elige el token más probable
  - Ventaja: Consistente, determinístico
  - Desventaja: Puede ser repetitivo
- `True` (sampling): Introduce aleatoriedad controlada
  - Ventaja: Más creativo y variado
  - Desventaja: Menos predecible

## Parte 2: Visualizando la Arquitectura del Modelo

Ahora vamos a "abrir" el modelo y ver su estructura interna.

In [4]:
# Mostramos la arquitectura completa
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_featur

### Componentes de Phi-3

En la salida anterior podés identificar:

1. **Phi3Model**: El modelo base
   - **embed_tokens**: Convierte IDs de tokens en vectores (embeddings)
   - **layers**: 32 capas Transformer idénticas
   - **norm**: Layer normalization final

2. **Phi3DecoderLayer** (cada capa contiene):
   - **self_attn**: Mecanismo de atención (Phi3Attention)
     - `q_proj`: Proyección de queries
     - `k_proj`: Proyección de keys
     - `v_proj`: Proyección de values
     - `o_proj`: Proyección de salida
   - **mlp**: Red feed-forward (Phi3MLP)
     - `gate_up_proj`: Proyección de entrada con activación
     - `down_proj`: Proyección de salida
   - **input_layernorm**: Normalización antes de atención
   - **post_attention_layernorm**: Normalización antes de MLP

3. **lm_head**: Capa final que convierte representaciones en probabilidades de vocabulario

### Flujo de Datos

```
Input IDs → Embeddings → Layer 1 → Layer 2 → ... → Layer 32 → Norm → LM Head → Logits
```

Cada capa procesa la información y la pasa a la siguiente, refinando progresivamente la representación.

## Parte 3: Proceso de Selección de Tokens

Vamos a ver cómo el modelo elige el siguiente token paso a paso.

In [5]:
import torch

# Prompt simple para demostración
prompt = "La capital de Argentina es"

# Tokenizamos
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")  # Movemos a GPU

print(f"Prompt: '{prompt}'")
print(f"Tokens: {input_ids}")
print(f"Cantidad de tokens: {input_ids.shape[1]}")

Prompt: 'La capital de Argentina es'
Tokens: tensor([[  997,  7483,   316, 13798,   831]], device='cuda:0')
Cantidad de tokens: 5


In [6]:
# Obtenemos la salida del modelo base (sin lm_head)
with torch.no_grad():  # No calculamos gradientes (solo inferencia)
    model_output = model.model(input_ids)

print("Salida del modelo base:")
print(f"Shape: {model_output[0].shape}")
print(f"Interpretación: [batch_size={model_output[0].shape[0]}, seq_length={model_output[0].shape[1]}, hidden_size={model_output[0].shape[2]}]")

Salida del modelo base:
Shape: torch.Size([1, 5, 3072])
Interpretación: [batch_size=1, seq_length=5, hidden_size=3072]


In [7]:
# Aplicamos la capa lm_head para obtener logits (scores sin normalizar)
lm_head_output = model.lm_head(model_output[0])

print("Salida de lm_head (logits):")
print(f"Shape: {lm_head_output.shape}")
print(f"Interpretación: [batch_size={lm_head_output.shape[0]}, seq_length={lm_head_output.shape[1]}, vocab_size={lm_head_output.shape[2]}]")
print(f"\nCada posición tiene {lm_head_output.shape[2]} scores, uno por cada token del vocabulario")

Salida de lm_head (logits):
Shape: torch.Size([1, 5, 32064])
Interpretación: [batch_size=1, seq_length=5, vocab_size=32064]

Cada posición tiene 32064 scores, uno por cada token del vocabulario


### Explicación de los Logits

Los **logits** son scores sin normalizar para cada token del vocabulario:
- Shape: `[1, seq_length, vocab_size]`
- Para predecir el siguiente token, tomamos la última posición: `logits[0, -1, :]`
- Valores más altos = mayor probabilidad de ser el siguiente token
- Se convierten a probabilidades con softmax: `P(token) = exp(logit) / sum(exp(all_logits))`

In [8]:
# Obtenemos el token con mayor probabilidad (greedy decoding)
next_token_logits = lm_head_output[0, -1, :]  # Última posición
next_token_id = next_token_logits.argmax(-1)  # argmax: Índice del valor más alto

next_token = tokenizer.decode(next_token_id)

print(f"Prompt: '{prompt}'")
print(f"Siguiente token predicho: '{next_token}'")
print(f"ID del token: {next_token_id.item()}")
print(f"Score (logit): {next_token_logits[next_token_id].item():.4f}")

Prompt: 'La capital de Argentina es'
Siguiente token predicho: 'Buenos'
ID del token: 17882
Score (logit): 38.0000


In [9]:
# Veamos los top 5 tokens más probables
import torch.nn.functional as F

# Convertimos logits a probabilidades
probabilities = F.softmax(next_token_logits, dim=-1)

# Top 5 tokens
top_k = 5
top_probs, top_indices = torch.topk(probabilities, top_k)

print(f"\nTop {top_k} tokens más probables después de '{prompt}':")
print("="*60)
for i, (prob, idx) in enumerate(zip(top_probs, top_indices), 1):
    token = tokenizer.decode(idx)
    print(f"{i}. '{token}' - Probabilidad: {prob.item():.4f} ({prob.item()*100:.2f}%)")


Top 5 tokens más probables después de 'La capital de Argentina es':
1. 'Buenos' - Probabilidad: 0.5859 (58.59%)
2. '_' - Probabilidad: 0.1021 (10.21%)
3. ':' - Probabilidad: 0.0481 (4.81%)
4. 'la' - Probabilidad: 0.0292 (2.92%)
5. '
' - Probabilidad: 0.0292 (2.92%)


### Estrategias de Decoding

**1. Greedy Decoding** (`do_sample=False`):
- Siempre elige el token con mayor probabilidad
- Determinístico: misma entrada → misma salida
- Puede ser repetitivo

**2. Sampling** (`do_sample=True`):
- Muestrea de la distribución de probabilidad
- Más variedad y creatividad
- Parámetros adicionales:
  - `temperature`: Controla aleatoriedad (0.1-2.0)
  - `top_k`: Solo considera los k tokens más probables
  - `top_p` (nucleus): Solo considera tokens cuya probabilidad acumulada sea ≤ p

**3. Beam Search**:
- Explora múltiples secuencias simultáneamente
- Elige la secuencia completa con mayor probabilidad total
- Más costoso computacionalmente

## Parte 4: Optimización con Caché KV

La generación autoregresiva es costosa porque cada token requiere procesar todo el contexto anterior. El **KV cache** optimiza esto.

### ¿Qué es el KV Cache?

En cada capa de atención, el modelo calcula:
- **Keys (K)**: Representaciones de tokens para ser "atendidos"
- **Values (V)**: Información a extraer de esos tokens
- **Queries (Q)**: Qué buscar en los otros tokens

**Sin caché:**
1. Token 1: Calcula K, V, Q para posición 1
2. Token 2: Recalcula K, V para posición 1, calcula nuevo K, V, Q para posición 2
3. Token 3: Recalcula K, V para posiciones 1-2, calcula nuevo K, V, Q para posición 3
4. ... (muchos cálculos redundantes)

**Con caché:**
1. Token 1: Calcula K, V, Q para posición 1, **guarda K, V**
2. Token 2: **Recupera K, V guardados**, solo calcula nuevo K, V, Q para posición 2
3. Token 3: **Recupera todos los K, V guardados**, solo calcula nuevo K, V, Q para posición 3

Ahorro: ~2x más rápido para secuencias largas, más ahorro con contextos más largos.

In [10]:
# Preparamos un prompt largo
prompt_largo = "Escribí una carta formal y detallada explicando por qué el asado del domingo se quemó. Incluí todos los detalles de qué salió mal y cómo sucedió paso a paso."

input_ids = tokenizer(prompt_largo, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

print(f"Longitud del prompt: {input_ids.shape[1]} tokens")
print(f"Vamos a generar 100 tokens adicionales")

Longitud del prompt: 55 tokens
Vamos a generar 100 tokens adicionales


In [11]:
%%timeit -n 1 -r 1
# Generación CON caché (default)
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=True          # use_cache: Activar KV cache (default)
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


9.53 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
%%timeit -n 1 -r 1
# Generación SIN caché (para comparación)
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=False         # Desactivar KV cache
)

38.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Análisis de Resultados

Deberías observar que `use_cache=True` es aproximadamente **2-3x más rápido** que `use_cache=False`.

**Trade-offs del KV Cache:**

**Ventajas:**
- Generación mucho más rápida
- Esencial para aplicaciones en tiempo real
- Más eficiente energéticamente

**Desventajas:**
- Usa más memoria (guarda K, V para cada capa y posición)
- Para contextos muy largos (miles de tokens), la memoria puede ser limitante
- En GPU con poca VRAM, puede causar OOM (Out Of Memory)

**Cuándo desactivar caché:**
- Cuando tenés memoria limitada
- Para debugging (comportamiento más simple)
- Nunca en producción (salvo casos muy específicos)

## Parte 5: Generando Texto Paso a Paso

Vamos a implementar manualmente el loop de generación para ver exactamente qué hace `model.generate()`.

In [13]:
def generar_paso_a_paso(prompt, max_tokens=20, mostrar_proceso=True):
    """
    Genera texto token por token, mostrando el proceso.

    Parámetros:
    - prompt: Texto inicial
    - max_tokens: Máximo de tokens a generar
    - mostrar_proceso: Si True, imprime cada paso
    """
    # Tokenizamos el prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

    if mostrar_proceso:
        print(f"Prompt inicial: '{prompt}'")
        print(f"Tokens iniciales: {input_ids.shape[1]}\n")
        print("="*80)

    # Loop de generación
    for i in range(max_tokens):
        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits

        # Obtenemos el siguiente token (greedy)
        next_token_logits = logits[0, -1, :]
        next_token_id = next_token_logits.argmax(-1).unsqueeze(0)

        # Decodificamos
        next_token = tokenizer.decode(next_token_id[0])

        if mostrar_proceso:
            print(f"Paso {i+1}: Generado '{next_token}' (ID: {next_token_id[0].item()})")

        # Añadimos el token a la secuencia
        input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim=1)

        # Condición de parada (si el modelo genera un token especial de fin)
        if next_token_id[0].item() == tokenizer.eos_token_id:
            if mostrar_proceso:
                print("\n[Modelo generó token de fin de secuencia]")
            break

    # Decodificamos toda la secuencia
    texto_completo = tokenizer.decode(input_ids[0])

    if mostrar_proceso:
        print("="*80)
        print(f"\nTexto completo generado:\n{texto_completo}")

    return texto_completo

# Probamos
prompt = "La capital de Argentina es"
resultado = generar_paso_a_paso(prompt, max_tokens=10)

Prompt inicial: 'La capital de Argentina es'
Tokens iniciales: 5

Paso 1: Generado 'Buenos' (ID: 17882)
Paso 2: Generado 'Aires' (ID: 16981)
Paso 3: Generado '.' (ID: 29889)
Paso 4: Generado '
' (ID: 13)
Paso 5: Generado '
' (ID: 13)
Paso 6: Generado '
' (ID: 13)
Paso 7: Generado '2' (ID: 29906)
Paso 8: Generado '.' (ID: 29889)
Paso 9: Generado '¿' (ID: 18613)
Paso 10: Generado 'C' (ID: 29907)

Texto completo generado:
La capital de Argentina es Buenos Aires.


2. ¿C


### Observaciones del Proceso

En la salida anterior podés ver:

1. **Cada token se genera secuencialmente**: El modelo no "planea" toda la respuesta de antemano
2. **Cada paso depende del anterior**: El modelo usa todo el contexto generado hasta ahora
3. **Token IDs vs texto**: Algunos tokens son subpalabras que se unen al decodificar
4. **Determinismo**: Con greedy decoding, siempre obtendrás la misma secuencia

Esta es exactamente la misma lógica que usa `model.generate()`, pero con optimizaciones adicionales (KV cache, batching, etc.).

## Resumen de Conceptos Clave

### Arquitectura Transformer

1. **Componentes principales**:
   - Embeddings: Convierte tokens en vectores
   - Self-Attention: Relaciona tokens entre sí
   - Feed-Forward: Procesa cada posición
   - Layer Norm: Estabiliza el entrenamiento

2. **Phi-3**: Modelo decoder-only de 32 capas con 3.8B parámetros

3. **Flujo de datos**: Input → Embeddings → 32 Layers → Norm → LM Head → Logits

### Generación de Texto

1. **Autoregresivo**: Genera un token a la vez, cada uno condicionado a los anteriores

2. **Logits**: Scores sin normalizar para cada token del vocabulario

3. **Decoding**:
   - Greedy: Siempre el más probable (determinístico)
   - Sampling: Introduce aleatoriedad (creativo)
   - Beam Search: Explora múltiples caminos

### Optimización

1. **KV Cache**: Guarda keys y values de atención para evitar recálculos
   - 2-3x más rápido para generación
   - Esencial para aplicaciones en tiempo real
   - Trade-off: Usa más memoria

2. **Parámetros importantes**:
   - `max_new_tokens`: Controla longitud de la salida
   - `use_cache`: Activa/desactiva KV cache
   - `do_sample`: Determinístico vs estocástico
   - `temperature`: Controla creatividad (con sampling)

## Guía de Preguntas y Respuestas

### Preguntas de Comprensión

**1. ¿Qué es el mecanismo de atención y por qué es fundamental en los Transformers?**

<details>
<summary>Ver respuesta</summary>

El **mecanismo de atención** permite que cada token en una secuencia "atienda" (observe y extraiga información) de otros tokens relevantes. Es fundamental porque:

1. **Captura dependencias a largo alcance**: Puede relacionar palabras separadas por muchas posiciones
2. **Procesamiento paralelo**: A diferencia de RNNs, procesa toda la secuencia simultáneamente
3. **Interpretabilidad**: Los pesos de atención muestran qué palabras el modelo considera relevantes
4. **Flexibilidad**: Se adapta dinámicamente según el contexto

Ejemplo: En "El banco del parque está roto", la atención ayuda al modelo a entender que "banco" se relaciona con "parque" y "está roto", no con conceptos financieros.
</details>

---

**2. ¿Cuál es la diferencia entre modelos encoder-only, decoder-only y encoder-decoder?**

<details>
<summary>Ver respuesta</summary>

**Encoder-only (BERT, RoBERTa)**:
- Procesa texto bidireccional (puede "ver" toda la oración)
- Ideal para: Clasificación, NER, sentiment analysis, QA extractivo
- No genera texto nuevo, solo analiza texto existente

**Decoder-only (GPT, Phi-3)**:
- Procesa texto unidireccional (izquierda a derecha)
- Ideal para: Generación de texto, completado, diálogo
- Genera texto nuevo token por token

**Encoder-Decoder (T5, BART)**:
- Encoder procesa la entrada, decoder genera la salida
- Ideal para: Traducción, summarización, parafraseo
- Transforma texto de una forma a otra

La elección depende de la tarea: comprensión → encoder, generación → decoder, transformación → encoder-decoder.
</details>

---

**3. ¿Qué son los logits y cómo se convierten en el próximo token?**

<details>
<summary>Ver respuesta</summary>

Los **logits** son scores sin normalizar que el modelo asigna a cada token del vocabulario para indicar qué tan probable es que sea el siguiente token.

Proceso:
1. **Modelo genera logits**: Un número por cada token del vocabulario (~32,000 para Phi-3)
2. **Softmax convierte a probabilidades**: `P(token_i) = exp(logit_i) / sum(exp(all_logits))`
3. **Selección del token**:
   - **Greedy**: Toma el token con mayor probabilidad
   - **Sampling**: Muestrea según la distribución de probabilidad
   - **Top-k**: Muestrea solo de los k más probables

Ejemplo simplificado:
```
Logits: ["Buenos": 8.5, "Ciudad": 2.1, "La": 1.3]
→ Softmax →
Probabilidades: ["Buenos": 0.95, "Ciudad": 0.03, "La": 0.02]
→ Greedy selecciona "Buenos"
```
</details>

---

**4. ¿Por qué la generación autoregresiva es secuencial y no se puede paralelizar completamente?**

<details>
<summary>Ver respuesta</summary>

La generación autoregresiva es inherentemente secuencial porque **cada token depende de todos los anteriores**:

```
Token 1: P(t1 | prompt)
Token 2: P(t2 | prompt, t1)         ← necesita t1
Token 3: P(t3 | prompt, t1, t2)     ← necesita t1 y t2
Token 4: P(t4 | prompt, t1, t2, t3) ← necesita t1, t2 y t3
```

No podés generar Token 3 sin antes haber generado Token 2, porque el modelo necesita saber qué dijiste antes para decidir qué decir después.

**Contraste con el entrenamiento**:
Durante el entrenamiento, el modelo SÍ procesa en paralelo porque ya tiene toda la secuencia (puede calcular pérdidas para todas las posiciones simultáneamente).

**Optimizaciones existentes**:
- KV cache: Reduce cálculos redundantes
- Speculative decoding: Predice varios tokens a la vez y verifica
- Distillation: Modelos más pequeños y rápidos
</details>

---

**5. ¿Qué es el KV cache y cómo funciona exactamente?**

<details>
<summary>Ver respuesta</summary>

El **KV cache** (Key-Value cache) es una técnica de optimización que guarda las representaciones de atención de tokens ya procesados.

**Mecánica de atención**:
Para cada token, la atención calcula:
- **Query (Q)**: "¿Qué estoy buscando?"
- **Key (K)**: "¿Qué información tengo?"
- **Value (V)**: "Información a extraer"

**Sin caché**:
```
Generar token 5:
  - Recalcular K, V para tokens 1-4 ← redundante
  - Calcular Q, K, V para token 5
  - Hacer atención
```

**Con caché**:
```
Generar token 5:
  - Recuperar K, V guardados de tokens 1-4 ← rápido
  - Solo calcular Q, K, V para token 5
  - Hacer atención
  - Guardar K, V del token 5
```

**Beneficio**: Evita recalcular keys y values de tokens anteriores en cada paso, ahorrando ~50-70% del tiempo de cómputo.

**Costo**: Memoria proporcional a `num_layers × num_heads × seq_length × hidden_dim`
</details>

---

### Preguntas de Aplicación

**6. Si tenés poca memoria GPU y el modelo da error OOM durante generación, ¿qué estrategias podrías usar?**

<details>
<summary>Ver respuesta</summary>

Estrategias para reducir uso de memoria:

1. **Desactivar KV cache**:
```python
model.generate(..., use_cache=False)
```
Más lento pero usa menos memoria.

2. **Reducir batch size** (si estás procesando múltiples ejemplos):
```python
model.generate(input_ids, max_new_tokens=100)  # Uno a la vez
```

3. **Acortar max_new_tokens**:
```python
model.generate(..., max_new_tokens=256)  # En lugar de 1024
```

4. **Usar modelo cuantizado** (menor precisión):
```python
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype=torch.float16,  # O torch.int8
    device_map="auto"
)
```

5. **Offloading a CPU**:
```python
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="auto",  # Distribuye automáticamente entre GPU/CPU
    offload_folder="offload"
)
```

6. **Modelo más pequeño**: Usar Phi-3-mini en lugar de modelos más grandes
</details>

---

**7. ¿Cómo implementarías sampling con temperature para hacer el modelo más creativo?**

<details>
<summary>Ver respuesta</summary>

```python
def generar_con_temperatura(prompt, temperature=1.0, max_tokens=50):
    """
    Genera texto con sampling y temperature.
    
    temperature:
    - < 1.0: Más conservador (ej: 0.5)
    - = 1.0: Sin modificación
    - > 1.0: Más aleatorio (ej: 1.5)
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    
    for _ in range(max_tokens):
        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits[0, -1, :]
        
        # Aplicar temperature
        logits = logits / temperature
        
        # Convertir a probabilidades
        probs = F.softmax(logits, dim=-1)
        
        # Sampling
        next_token_id = torch.multinomial(probs, num_samples=1)
        
        # Añadir a la secuencia
        input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim=1)
        
        if next_token_id.item() == tokenizer.eos_token_id:
            break
    
    return tokenizer.decode(input_ids[0])

# Probar con diferentes temperaturas
prompt = "Había una vez"
print("Temperature 0.5 (conservador):")
print(generar_con_temperatura(prompt, temperature=0.5))
print("\nTemperature 1.5 (creativo):")
print(generar_con_temperatura(prompt, temperature=1.5))
```

O simplemente:
```python
model.generate(
    input_ids,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.8
)
```
</details>

---

**8. Implementá Top-K sampling: solo considera los K tokens más probables antes de hacer sampling.**

<details>
<summary>Ver respuesta</summary>

```python
def generar_top_k(prompt, k=50, temperature=1.0, max_tokens=50):
    """
    Sampling considerando solo los top-k tokens más probables.
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    
    for _ in range(max_tokens):
        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits[0, -1, :]
        
        # Aplicar temperature
        logits = logits / temperature
        
        # Top-K filtering
        top_k_logits, top_k_indices = torch.topk(logits, k)
        
        # Crear distribución solo con top-k
        probs = F.softmax(top_k_logits, dim=-1)
        
        # Sampling del top-k
        sampled_index = torch.multinomial(probs, num_samples=1)
        next_token_id = top_k_indices[sampled_index]
        
        # Añadir a la secuencia
        input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim=1)
        
        if next_token_id.item() == tokenizer.eos_token_id:
            break
    
    return tokenizer.decode(input_ids[0])

# Probar
print(generar_top_k("Había una vez", k=10))  # Muy restrictivo
print(generar_top_k("Había una vez", k=100)) # Más variado
```

O con Hugging Face:
```python
model.generate(
    input_ids,
    do_sample=True,
    top_k=50,
    temperature=0.8
)
```
</details>

---

**9. ¿Cómo medirías cuánta memoria consume el KV cache para una secuencia de 1000 tokens?**

<details>
<summary>Ver respuesta</summary>

```python
import torch

def calcular_memoria_kv_cache(modelo, seq_length=1000):
    """
    Estima la memoria usada por KV cache.
    """
    # Parámetros del modelo
    num_layers = modelo.config.num_hidden_layers
    num_heads = modelo.config.num_attention_heads
    hidden_size = modelo.config.hidden_size
    head_dim = hidden_size // num_heads
    
    # Tamaño de K y V por capa
    # Shape: [batch_size, num_heads, seq_length, head_dim]
    k_size_per_layer = 1 * num_heads * seq_length * head_dim
    v_size_per_layer = 1 * num_heads * seq_length * head_dim
    
    # Total para todas las capas (K + V)
    total_elements = (k_size_per_layer + v_size_per_layer) * num_layers
    
    # Memoria en bytes (float16 = 2 bytes)
    bytes_per_element = 2  # float16
    total_bytes = total_elements * bytes_per_element
    
    # Convertir a MB/GB
    mb = total_bytes / (1024 ** 2)
    gb = total_bytes / (1024 ** 3)
    
    print(f"KV Cache para secuencia de {seq_length} tokens:")
    print(f"  Capas: {num_layers}")
    print(f"  Attention heads: {num_heads}")
    print(f"  Hidden size: {hidden_size}")
    print(f"  Head dim: {head_dim}")
    print(f"\n  Memoria KV cache: {mb:.2f} MB ({gb:.3f} GB)")
    
    return mb

# Calcular para Phi-3
calcular_memoria_kv_cache(model, seq_length=1000)
calcular_memoria_kv_cache(model, seq_length=4000)  # Contexto completo
```

Para medir memoria real:
```python
# Antes de generar
torch.cuda.reset_peak_memory_stats()
memoria_inicial = torch.cuda.memory_allocated() / 1024**2

# Generar con caché
output = model.generate(input_ids, max_new_tokens=1000, use_cache=True)

# Después de generar
memoria_pico = torch.cuda.max_memory_allocated() / 1024**2
print(f"Memoria adicional usada: {memoria_pico - memoria_inicial:.2f} MB")
```
</details>

---

**10. ¿Cómo implementarías beam search con beam_size=3?**

<details>
<summary>Ver respuesta</summary>

Beam search mantiene las top-N secuencias más probables en cada paso:

```python
def beam_search(prompt, beam_size=3, max_tokens=20):
    """
    Beam search simple.
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    
    # Inicializar beams: (secuencia, score_acumulado)
    beams = [(input_ids, 0.0)]
    
    for _ in range(max_tokens):
        all_candidates = []
        
        # Para cada beam actual
        for seq, score in beams:
            with torch.no_grad():
                outputs = model(seq)
                logits = outputs.logits[0, -1, :]
                log_probs = F.log_softmax(logits, dim=-1)
            
            # Top-k candidatos
            top_log_probs, top_indices = torch.topk(log_probs, beam_size)
            
            # Crear nuevos candidatos
            for log_prob, idx in zip(top_log_probs, top_indices):
                new_seq = torch.cat([seq, idx.unsqueeze(0).unsqueeze(0)], dim=1)
                new_score = score + log_prob.item()
                all_candidates.append((new_seq, new_score))
        
        # Seleccionar top beam_size candidatos
        beams = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_size]
    
    # Devolver el mejor beam
    best_seq, best_score = beams[0]
    return tokenizer.decode(best_seq[0])

# Probar
print(beam_search("La capital de Argentina es", beam_size=3))
```

O usar el método integrado:
```python
output = model.generate(
    input_ids,
    max_new_tokens=20,
    num_beams=3,
    early_stopping=True
)
```
</details>

---

### Desafíos Adicionales

**Desafío 1**: Implementá un contador que muestre cuántas veces se ejecuta el forward pass del modelo durante la generación de 100 tokens, con y sin KV cache.

**Desafío 2**: Creá un visualizador que muestre, en cada paso de generación, los top 3 tokens candidatos y sus probabilidades.

**Desafío 3**: Implementá nucleus sampling (Top-P): en lugar de top-K, incluí los tokens más probables cuya probabilidad acumulada sea ≤ P.

---

## Referencias y Recursos Adicionales

- [Attention is All You Need (Paper original)](https://arxiv.org/abs/1706.03762)
- [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)
- [Hands-On Large Language Models (O'Reilly)](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961)
- [Phi-3 Technical Report](https://arxiv.org/abs/2404.14219)
- [Hugging Face Generation Documentation](https://huggingface.co/docs/transformers/main_classes/text_generation)